In [2]:
import os
import json
import random
from openai import OpenAI
from datetime import datetime

In [3]:
OPENAI_API_KEY = "sk-bbmDxDlAMbtbL6THJu7mo0gTmGbgI7yGUOPtAWHdIAT3BlbkFJo-sBd6zx_q4TfX9WMDGNgMiUmRkWP9BW7NYLhY5NkA"
NUM_EXAMPLES = 9
MAX_VALIDATE = 1
timestmp=datetime.now().strftime('%Y%m%d_%H%M%S')
RESULTS_PATH = f'{NUM_EXAMPLES}-SHOT_{timestmp}'
os.mkdir(RESULTS_PATH)
data_filepath = './data.json'

In [4]:
# ---------------- helpers ------------------ #
def write_to_disk(data, filename): 
    path = f'{RESULTS_PATH}/{filename}.json'
    with open(path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def create_messages(examples):
    messages = []
    for sample in examples:
        messages.extend([
            {
                "role": "user",
                "content": [
                    {
                        "text": sample['dispositivo'],
                        "type": "text"
                    }
                ]
            },
            {
                "role": "assistant",
                "content": [
                    {
                        "text": json.dumps(sample['decisoes']),
                        "type": "text"
                    }
                ]
            }
        ])
    return messages

def get_completion(client, examples, dispositivo): 
    sys_message = [{
        "role": "system",
        "content": [
            {
                "text": "Você é um assistente judicial que identifica e classifica as decisões contidas em uma sentença judicial. Cada decisão diz respeito a um pedido do autor.",
                "type": "text"
            }
        ]
    }]
    example_messages = create_messages(examples)
    current_message = [{
        "role": "user",
        "content": [
            {
                "text": dispositivo,
                "type": "text"
            }
        ]
    }]
    messages = sys_message + example_messages + current_message

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "decisao_judicial",
                "schema": {
                    "type": "object",
                    "required": [
                    "decisoes"
                    ],
                    "properties": {
                    "decisoes": {
                        "type": "array",
                        "items": {
                        "type": "object",
                        "required": [
                            "pedido",
                            "tipoDaDecisao",
                            "textoDaDecisao",
                            "valorDeferido",
                            "obrigacaoDeFazer"
                        ],
                        "properties": {
                            "pedido": {
                            "enum": [
                                "Danos morais",
                                "Danos materiais",
                                "Obrigação de fazer",
                                "Declaratória",
                                "Gratuidade da justiça"
                            ],
                            "type": "string",
                            "title": "Pedido"
                            },
                            "tipoDaDecisao": {
                            "enum": [
                                "Procedente",
                                "Improcedente",
                                "Parcialmente procedente",
                                "Não mencionado"
                            ],
                            "type": "string"
                            },
                            "valorDeferido": {
                            "type": "number"
                            },
                            "textoDaDecisao": {
                            "type": "string"
                            },
                            "obrigacaoDeFazer": {
                            "type": "string",
                            "title": "Descrição da obrigação de fazer, se for o caso"
                            }
                        },
                        "additionalProperties": False
                        }
                    }
                    },
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )
    decisoes = response.choices[0].message.content
    return response


In [12]:
client = OpenAI(
    api_key=OPENAI_API_KEY
)



In [15]:
# load data 
with open(data_filepath, 'r', encoding='utf-8') as file:
    data = json.load(file)
    
rand_sample = random.choice(data)
data.remove(rand_sample)
examples = [rand_sample]
rand_sample = random.choice(data)
data = [rand_sample]

In [16]:
response = get_completion(client, examples, data[0]['dispositivo'])
print(response)


ChatCompletion(id='chatcmpl-9xhQyoeBhWvc7KKWJIaY9r7lbvjmS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"decisoes":[{"pedido":"Declaratória","tipoDaDecisao":"Procedente","valorDeferido":0,"textoDaDecisao":"Julgo procedente a ação, para consolidar a propriedade e a posse exclusiva do bem no patrimônio do(a) autor(a), tornando definitiva a liminar.","obrigacaoDeFazer":""}]}', role='assistant', function_call=None, tool_calls=None, refusal=None))], created=1724015340, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_48196bc67a', usage=CompletionUsage(completion_tokens=75, prompt_tokens=415, total_tokens=490))


In [26]:
log_data = {
    'id': response.id,
    'choices': [
        {
            'finish_reason': choice.finish_reason,
            'index': choice.index,
            'message_content': choice.message.content,
            'message_role': choice.message.role,
        }
        for choice in response.choices
    ],
    'created': response.created,
    'model': response.model,
    'object': response.object,
    'usage': {
        'completion_tokens': response.usage.completion_tokens,
        'prompt_tokens': response.usage.prompt_tokens,
        'total_tokens': response.usage.total_tokens,
    }
}

print(log_data)

{'id': 'chatcmpl-9xhQyoeBhWvc7KKWJIaY9r7lbvjmS', 'choices': [{'finish_reason': 'stop', 'index': 0, 'message_content': '{"decisoes":[{"pedido":"Declaratória","tipoDaDecisao":"Procedente","valorDeferido":0,"textoDaDecisao":"Julgo procedente a ação, para consolidar a propriedade e a posse exclusiva do bem no patrimônio do(a) autor(a), tornando definitiva a liminar.","obrigacaoDeFazer":""}]}', 'message_role': 'assistant'}], 'created': 1724015340, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion', 'usage': {'completion_tokens': 75, 'prompt_tokens': 415, 'total_tokens': 490}}


In [34]:
decisoes = json.loads(response.choices[0].message.content)
result = {
    "id": 1123,
    **decisoes
}
print(result)

{'id': 1123, 'decisoes': [{'pedido': 'Declaratória', 'tipoDaDecisao': 'Procedente', 'valorDeferido': 0, 'textoDaDecisao': 'Julgo procedente a ação, para consolidar a propriedade e a posse exclusiva do bem no patrimônio do(a) autor(a), tornando definitiva a liminar.', 'obrigacaoDeFazer': ''}]}
